In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Step 1: Install necessary libraries
!pip install tensorflow



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = '/content/drive/MyDrive/Colab_Notebooks/gaussian/train'
val_dir = '/content/drive/MyDrive/Colab_Notebooks/gaussian/val'
target_size = (224, 224)
batch_size = 32


# New Section

In [ ]:

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 20 images belonging to 2 classes.


In [ ]:

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 4 images belonging to 2 classes.


In [ ]:

# Step 3: Load the pre-trained ResNet50 model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

num_classes = len(train_generator.class_indices)

base_model = ResNet50(weights='imagenet', include_top=False)


In [ ]:

# Step 4: Fine-tune the model for your dataset
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False


In [ ]:

# Step 5: Train the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

epochs = 4
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator
)


Epoch 1/4
1/1 [==============================] - 12s 12s/step - loss: 0.7859 - accuracy: 0.5000 - val_loss: 2.4762 - val_accuracy: 0.5000
Epoch 2/4
1/1 [==============================] - 5s 5s/step - loss: 2.4879 - accuracy: 0.5000 - val_loss: 1.3694 - val_accuracy: 0.5000
Epoch 3/4
1/1 [==============================] - 5s 5s/step - loss: 1.3685 - accuracy: 0.5000 - val_loss: 0.7957 - val_accuracy: 0.5000
Epoch 4/4
1/1 [==============================] - 7s 7s/step - loss: 0.7934 - accuracy: 0.5000 - val_loss: 1.3782 - val_accuracy: 0.5000


In [ ]:
# Step 6: Predict on new images
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

def predict_image(image_path):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.

    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)

    return predicted_class

image_path = '/content/drive/MyDrive/Colab Notebooks/Colab_Notebooks/person1946_bacteria_4874.jpeg'
predicted_class = predict_image(image_path)
print("Predicted class:", predicted_class)


1/1 [==============================] - 0s 237ms/step
Predicted class: [1]
